In [1]:
!ls output/ensembletr_output/
!mkdir -p downstream

advntr_strs.vcf.gz	ensembletr.vcf.gz	 hipstr_strs.vcf.gz
advntr_strs.vcf.gz.tbi	ensembletr.vcf.gz.tbi	 hipstr_strs.vcf.gz.tbi
eh_strs.vcf.gz		gangstr_strs.vcf.gz
eh_strs.vcf.gz.tbi	gangstr_strs.vcf.gz.tbi


In [2]:
# pip install sgkit
# pip install sgkit[vcf]
# pip install cyvcf2

In [3]:
# import modules
import sgkit as sg
 
from sgkit.io.vcf import vcf_to_zarr
from cyvcf2 import VCF

import xarray as xr
xr.set_options(display_expand_attrs=False, display_expand_data_vars=True)
import zarr

import pandas as pd 
import numpy as np 

In [5]:
# view VCF header
!zgrep "##" output/ensembletr_output/ensembletr.vcf.gz

##fileformat=VCFv4.1
##command=/home/ibra/.local/bin/EnsembleTR --ref references/reference_chroms.fa --vcfs output/ensembletr_output/advntr_strs.vcf.gz,output/ensembletr_output/eh_strs.vcf.gz,output/ensembletr_output/gangstr_strs.vcf.gz,output/ensembletr_output/hipstr_strs.vcf.gz --out output/ensembletr_output/ensembletr.vcf
##INFO=<ID=START,Number=1,Type=Integer,Description="First position in all alleles">
##INFO=<ID=END,Number=1,Type=Integer,Description="Last position in all alleles">
##INFO=<ID=PERIOD,Number=1,Type=Integer,Description="Length of motif (repeat unit)">
##INFO=<ID=RU,Number=1,Type=String,Description="Motif (repeat unit)">
##INFO=<ID=METHODS,Number=1,Type=String,Description="Methods that attempted to genotype this locus (AdVNTR, EH, HipSTR, GangSTR)">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=GB,Number=1,Type=String,Description="Base pair difference from ref allele">
##FORMAT=<ID=NCOPY,Number=1,Type=String,Description="Copy Number">
##FOR

## For the tutorial

For large studies with a large number of samples and data, it is recommended to use per chromosome-VCFs which are converted to per-chromosome zarrs

In [7]:
# parse VCF to zarr format
# for very large files, it is advisable to have a per-chromosome VCF file that 
# is bgzipped and indexed
# see here: https://github.com/pystatgen/sgkit/issues/725

vcf_to_zarr("output/ensembletr_output/ensembletr.vcf.gz", "downstream/strs.zarr",
        max_alt_alleles = 200, #should this be 2N
                fields = ["INFO/START","INFO/END","INFO/PERIOD","INFO/RU","INFO/METHODS",
                        "FORMAT/GT","FORMAT/GB", "FORMAT/NCOPY", "FORMAT/EXP", 
                        "FORMAT/SCORE","FORMAT/GTS", "FORMAT/ALS", "FORMAT/INPUTS"],
                #     field_defs = {
                #     "INFO/START" : {"Number":"R"},
                #     "INFO/END" : {"Number":"R"},
                #     "INFO/PERIOD" : {"Number":"R"},
                #     "INFO/RU" : {"String":"R"},
                #     "INFO/METHODS" : {"String":"."},
                #     "FORMAT/GT" : {"Number": "G"},
                #     "FORMAT/GB" : {"Number": "A"},
                #     "FORMAT/NCOPY" : {"Number": "A"},
                #     "FORMAT/EXP" : {"Number": "."},
                #     "FORMAT/SCORE" : {"Number": "."},
                #     "FORMAT/GTS" : {"String": "."},
                #     "FORMAT/ALS" : {"String": "."},
                #     "FORMAT/INPUTS" : {"String": "."}}
                )
ds = sg.load_dataset("downstream/strs.zarr")
ds

/home/ibra/miniconda3/lib/python3.11/site-packages/sgkit/io/vcf/vcf_reader.py:574: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ds.dims["samples"] > 0:
/home/ibra/miniconda3/lib/python3.11/site-packages/sgkit/io/vcf/vcf_reader.py:575: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunk_width = min(chunk_width, ds.dims["samples"])
/home/ibra/miniconda3/lib/python3.11/site-packages/sgkit/io/vcf/vcf_reader.py:599: FloatFormatFieldWarning: Storing call variable call_SCORE (FORMAT field) as a float can result in large file sizes. Consider setting the encoding filters for this variable to Fi

<xarray.Dataset> Size: 54kB
Dimensions:               (variants: 7, samples: 100, ploidy: 2, contigs: 6,
                           filters: 1, alleles: 201)
Dimensions without coordinates: variants, samples, ploidy, contigs, filters,
                                alleles
Data variables: (12/25)
    call_ALS              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_EXP              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_GB               (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_GTS              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_INPUTS           (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_NCOPY            (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    ...                    ...
    variant_contig        (variants) int8 7B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_filter        (variants, filters) bool 7B dask.array<chunksize=(7, 1), meta=np.ndarray>
    variant_id            (variants) object 56B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_id_mask       (variants) bool 7B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_position      (variants) int32 28B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_quality       (variants) float32 28B dask.array<chunksize=(7,), meta=np.ndarray>
Attributes: (6)

In [8]:
# add metadata
metadata_df = pd.read_csv("str_resources/metadata.csv", index_col="Sample")
ds_annotations = pd.DataFrame.to_xarray(metadata_df).rename({"Sample":"samples"})
ds = ds.set_index({"samples": "sample_id"})

ds = ds.merge(ds_annotations, join="left")
ds = ds.reset_index("samples").reset_coords(drop=True)
ds

<xarray.Dataset> Size: 55kB
Dimensions:               (samples: 100, variants: 7, ploidy: 2, contigs: 6,
                           filters: 1, alleles: 201)
Dimensions without coordinates: samples, variants, ploidy, contigs, filters,
                                alleles
Data variables: (12/26)
    call_ALS              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_EXP              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_GB               (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_GTS              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_INPUTS           (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_NCOPY            (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    ...                    ...
    variant_id            (variants) object 56B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_id_mask       (variants) bool 7B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_position      (variants) int32 28B dask.array<chunksize=(7,), meta=np.ndarray>
    variant_quality       (variants) float32 28B dask.array<chunksize=(7,), meta=np.ndarray>
    sex                   (samples) object 800B 'F' 'M' 'M' 'M' ... 'M' 'F' 'F'
    super_population      (samples) object 800B 'America' ... 'South Asia'
Attributes: (6)

In [82]:
ds_annotations.super_population.to_series().value_counts()

super_population
Europe        20
East Asia     20
America       20
Africa        20
South Asia    20
Name: count, dtype: int64

In [84]:
ds_annotations.sex.to_series().value_counts()

sex
F    51
M    49
Name: count, dtype: int64

In [39]:
ds.data_vars

Data variables:
    call_ALS              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_EXP              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_GB               (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_GTS              (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_INPUTS           (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_NCOPY            (variants, samples) object 6kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_SCORE            (variants, samples) float32 3kB dask.array<chunksize=(7, 100), meta=np.ndarray>
    call_genotype         (variants, samples, ploidy) int16 3kB dask.array<chunksize=(7, 100, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool 1kB dask.array<chunksize=(7, 100, 2), meta=np.ndarray>
    call_genotype_p

In [51]:
vars = ['call_genotype','call_SCORE','call_NCOPY','call_EXP','call_GB','variant_contig','variant_allele','sex','super_population',]
df_variant = ds[[v for v in ds.data_vars if v in vars]].to_dataframe().reset_index(drop=True)

# remove rows with empty alleles
df_variant = df_variant.loc[df_variant['variant_allele'] != '']

In [52]:
df_variant

,call_EXP,call_GB,call_NCOPY,call_SCORE,call_genotype,variant_allele,variant_contig,sex,super_population
0,.,.,.,NaN,-1,TTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTC...,chr1,F,America
1,.,.,.,NaN,-1,T,chr1,F,America
201,.,.,.,NaN,-1,TTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTC...,chr1,F,America
202,.,.,.,NaN,-1,T,chr1,F,America
402,1/1,-96/-96,"0.0,0.0",0.0,1,TTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTC...,chr1,M,Africa
...,...,...,...,...,...,...,...,...,...
281001,1/1,-24/-24,"1.0,1.0",0.0,2,G,chr21,F,South Asia
281199,1/1,-24/-24,"1.0,1.0",0.0,2,GCGCGGGGCGGGGCGCGGGGCGGGGCGCGGGGCGGGG,chr21,F,South Asia
281200,1/1,-24/-24,"1.0,1.0",0.0,2,GCGCGGGGCGGGGCGCGGGGCGGGG,chr21,F,South Asia
281201,1/1,-24/-24,"1.0,1.0",0.0,2,GCGCGGGGCGGGG,chr21,F,South Asia


In [14]:
df_variant = ds[[v for v in ds.data_vars if v.startswith("variant_")]].to_dataframe()
df_variant.groupby(["variant_contig", "variant_position", "variant_id"]).agg({"variant_allele": lambda x: list(x)}).head(5)

variant_allele
variant_contig variant_position variant_id                                                   
0              77887913         .           [TTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTT...
1              3074877          .           [CAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAG...
               3074940          .           [CCGCCGCCGCCGCCGCCGCCGCCGCCG, CCGCCGCCGCCGCCGC...
2              57528484         .           [TATATATATATATATATATATATATATATATATA, TATATATAT...
3              64253561         .           [TGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGT...

In [ ]:
# summarising function that get ref_allele, alt alleles, their length, and samples and population for each
# email Helya for code used downstream if it fails

In [38]:
# number of alt alleles per STR
chr4_alleles_arr = df_variant.loc[[1]][["variant_allele"]].to_numpy()
chr4_alleles_list = ["".join(row) for row in chr4_alleles_arr.flatten() if row]
print(len(chr4_alleles_list))
[len(allele) for allele in chr4_alleles_list]


42


[92,
 80,
 86,
 95,
 101,
 95,
 71,
 92,
 92,
 83,
 98,
 92,
 92,
 86,
 98,
 89,
 89,
 104,
 86,
 110,
 107,
 101,
 83,
 89,
 95,
 86,
 104,
 107,
 89,
 89,
 116,
 101,
 98,
 80,
 71,
 74,
 80,
 129,
 89,
 96,
 105,
 95]

In [17]:
# inspect alleles
ds["variant_contig_name"] = ds.contig_id[ds.variant_contig]

ds2 = ds.set_index({"variants": ("variant_contig_name","variant_position","variant_id")})
sg.display_genotypes(ds2, max_variants=20, max_samples=5)


TypeError: invalid indexer array, does not have integer dtype: dask.array<slice_with_int_dask_array_aggregate, shape=(7,), dtype=<U5, chunksize=(7,), chunktype=numpy.ndarray>

In [26]:
df_variant = ds[[v for v in ds.data_vars if v.startswith("variant_")]].to_dataframe()
df_variant.groupby(["variant_contig_name","variant_position","variant_id"]).agg({"variant_allele": lambda x: list(x)})

variant_allele
variant_contig_name variant_position variant_id                                                   
chr1                77887913         .           [TTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTT...
chr11               57528484         .           [TATATATATATATATATATATATATATATATATA, TATATATAT...
chr14               64253561         .           [TGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGT...
chr17               51831668         .           [AGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGC...
chr21               43776443         .           [GCGCGGGGCGGGGCGCGGGGCGGGGCGCGGGGCGGGG, GCGCGG...
chr4                3074877          .           [CAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAGCAG...
                    3074940          .           [CCGCCGCCGCCGCCGCCGCCGCCGCCG, CCGCCGCCGCCGCCGC...